In [3]:
import pickle
import torch
import pandas as pd
import sys
import os
import lightning as pl
import numpy as np
import torch_geometric
from pathlib import Path

sys.path.insert(0, str(Path.cwd().resolve().parent))
from data import KIRCDataset #,KIRCDataModule

from torch_geometric.explain import Explainer, GNNExplainer
from tqdm import tqdm
from gnn_models import GCN, GCN2MLP, GAT, GAT2MLP, ChebNet, ChebNet2MLP
import scipy
import matplotlib.pyplot as plt



In [4]:
list_GNN_layers= ['first_layer.lins.0.weight',
                  'first_layer.lins.1.weight',
                  'first_layer.lins.2.weight',
                  'first_layer.lins.3.weight',
                  'first_layer.lins.4.weight',
                  'first_layer.lins.5.weight',
                  'first_layer.lins.6.weight',
                  'first_layer.lins.7.weight',
                  'hidden_layers.0.lins.0.weight',
                  'hidden_layers.0.lins.1.weight',
                  'hidden_layers.0.lins.2.weight',
                  'hidden_layers.0.lins.3.weight',
                  'hidden_layers.0.lins.4.weight',
                  'hidden_layers.0.lins.5.weight',
                  'hidden_layers.0.lins.6.weight',
                  'hidden_layers.0.lins.7.weight']                  

In [5]:
def analysis_checkpoint(checkpoint):

    min_val_gnn = sys.float_info.max
    max_val_gnn = sys.float_info.min
    mean_val_gnn = 0
    std_val_gnn = 0

    for layer in list_GNN_layers:
        min_val_gnn = min(min_val_gnn, checkpoint['state_dict'][layer].numpy().min())
        max_val_gnn = max(max_val_gnn, checkpoint['state_dict'][layer].numpy().max())
        mean_val_gnn += checkpoint['state_dict'][layer].numpy().mean()
        std_val_gnn += checkpoint['state_dict'][layer].numpy().std()

    mean_val_gnn = mean_val_gnn/len(list_GNN_layers)
    std_val_gnn = std_val_gnn/len(list_GNN_layers)

    print('MIN WEIGHT GNN LAYERS (ALL): ', min_val_gnn)
    print('MAX WEIGHT GNN LAYERS (ALL): ', max_val_gnn)
    print('MEAN WEIGHT GNN LAYERS (ALL): ', mean_val_gnn)
    print('STD WEIGHT GNN LAYERS (ALL): ', std_val_gnn)


    min_val_mlp = checkpoint['state_dict']['linear_last.weight'].numpy().min()
    max_val_mlp = checkpoint['state_dict']['linear_last.weight'].numpy().max()
    mean_val_mlp = checkpoint['state_dict']['linear_last.weight'].numpy().mean()
    std_val_mlp = checkpoint['state_dict']['linear_last.weight'].numpy().std()

    print('MIN WEIGHT MLP LAYER: ', min_val_mlp)
    print('MAX WEIGHT MLP LAYER: ', max_val_mlp)
    print('MEAN WEIGHT MLP LAYER: ', mean_val_mlp)
    print('STD WEIGHT MLP LAYER: ', std_val_mlp)


In [6]:
#path = '/Users/julia/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet' #old
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet'
files = os.listdir(path)

print('FOR CHEBNET: \n')

for path, subdirs, files in os.walk(path):
    for name in files:
        file_path = os.path.join(path, name).replace("\\","/")
        ckpt = torch.load(file_path, map_location=torch.device('cpu'))

        print(path[91:102], path[-11:-6])
        analysis_checkpoint(ckpt)
        print('\n')

FOR CHEBNET: 

test_round0 fold0
MIN WEIGHT GNN LAYERS (ALL):  -0.88695234
MAX WEIGHT GNN LAYERS (ALL):  0.72219896
MEAN WEIGHT GNN LAYERS (ALL):  -0.0006648341
STD WEIGHT GNN LAYERS (ALL):  0.09807233
MIN WEIGHT MLP LAYER:  -0.5520477
MAX WEIGHT MLP LAYER:  1.2708386
MEAN WEIGHT MLP LAYER:  0.16809894
STD WEIGHT MLP LAYER:  0.4266039


test_round0 fold1
MIN WEIGHT GNN LAYERS (ALL):  -1.0222968
MAX WEIGHT GNN LAYERS (ALL):  0.84385747
MEAN WEIGHT GNN LAYERS (ALL):  -0.0016772833
STD WEIGHT GNN LAYERS (ALL):  0.09690009
MIN WEIGHT MLP LAYER:  -1.092822
MAX WEIGHT MLP LAYER:  1.4945786
MEAN WEIGHT MLP LAYER:  0.16909231
STD WEIGHT MLP LAYER:  0.5177148


test_round0 fold2
MIN WEIGHT GNN LAYERS (ALL):  -1.1108353
MAX WEIGHT GNN LAYERS (ALL):  1.0130012
MEAN WEIGHT GNN LAYERS (ALL):  -0.0021803887
STD WEIGHT GNN LAYERS (ALL):  0.0926789
MIN WEIGHT MLP LAYER:  -1.5041121
MAX WEIGHT MLP LAYER:  2.1975713
MEAN WEIGHT MLP LAYER:  0.118366644
STD WEIGHT MLP LAYER:  0.53292143


test_round0 fold

C:\Users\schirmacher3\AppData\Local\Temp\ipykernel_22812\528192847.py:9: RuntimeWarning: overflow encountered in cast
  min_val_gnn = min(min_val_gnn, checkpoint['state_dict'][layer].numpy().min())


test_round1 fold3
MIN WEIGHT GNN LAYERS (ALL):  -2.6376379
MAX WEIGHT GNN LAYERS (ALL):  2.5956
MEAN WEIGHT GNN LAYERS (ALL):  0.008810609
STD WEIGHT GNN LAYERS (ALL):  0.3175195
MIN WEIGHT MLP LAYER:  -1.3472309
MAX WEIGHT MLP LAYER:  0.82219034
MEAN WEIGHT MLP LAYER:  -0.10068789
STD WEIGHT MLP LAYER:  0.58225006


test_round1 fold4
MIN WEIGHT GNN LAYERS (ALL):  -3.5021062
MAX WEIGHT GNN LAYERS (ALL):  3.8287392
MEAN WEIGHT GNN LAYERS (ALL):  0.010540723
STD WEIGHT GNN LAYERS (ALL):  0.3391332
MIN WEIGHT MLP LAYER:  -1.8671964
MAX WEIGHT MLP LAYER:  0.96372586
MEAN WEIGHT MLP LAYER:  -0.13009655
STD WEIGHT MLP LAYER:  0.60113454


test_round2 fold0
MIN WEIGHT GNN LAYERS (ALL):  -4.3686104
MAX WEIGHT GNN LAYERS (ALL):  3.73933
MEAN WEIGHT GNN LAYERS (ALL):  0.009841952
STD WEIGHT GNN LAYERS (ALL):  0.35444802
MIN WEIGHT MLP LAYER:  -1.4589856
MAX WEIGHT MLP LAYER:  0.74462813
MEAN WEIGHT MLP LAYER:  -0.22357437
STD WEIGHT MLP LAYER:  0.59904814


test_round2 fold1
MIN WEIGHT GNN LAYER

In [7]:
#path = '/Users/julia/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet2MLP' #old
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet2MLP'

print('FOR CHEBNET2MLP: \n')

for path, subdirs, files in os.walk(path):
    for name in files:
        file_path = os.path.join(path, name).replace("\\","/")
        ckpt = torch.load(file_path, map_location=torch.device('cpu'))

        print(path[96:102], path[-11:-6])
        analysis_checkpoint(ckpt)
        print('\n')

FOR CHEBNET2MLP: 

est_ro fold0
MIN WEIGHT GNN LAYERS (ALL):  -0.37319222
MAX WEIGHT GNN LAYERS (ALL):  0.37488878
MEAN WEIGHT GNN LAYERS (ALL):  -0.00024140172
STD WEIGHT GNN LAYERS (ALL):  0.13748427
MIN WEIGHT MLP LAYER:  -0.1392816
MAX WEIGHT MLP LAYER:  0.12498368
MEAN WEIGHT MLP LAYER:  0.0016146939
STD WEIGHT MLP LAYER:  0.03433848


est_ro fold1
MIN WEIGHT GNN LAYERS (ALL):  -0.3723457
MAX WEIGHT GNN LAYERS (ALL):  0.34679914
MEAN WEIGHT GNN LAYERS (ALL):  0.000112427864
STD WEIGHT GNN LAYERS (ALL):  0.14339459
MIN WEIGHT MLP LAYER:  -0.10230258
MAX WEIGHT MLP LAYER:  0.093100525
MEAN WEIGHT MLP LAYER:  -0.0049294797
STD WEIGHT MLP LAYER:  0.031395778


est_ro fold2
MIN WEIGHT GNN LAYERS (ALL):  -0.4607635
MAX WEIGHT GNN LAYERS (ALL):  0.38964167
MEAN WEIGHT GNN LAYERS (ALL):  -0.0027223486
STD WEIGHT GNN LAYERS (ALL):  0.14156061
MIN WEIGHT MLP LAYER:  -0.31273174
MAX WEIGHT MLP LAYER:  0.1357513
MEAN WEIGHT MLP LAYER:  -0.0071166116
STD WEIGHT MLP LAYER:  0.04192708


est_ro 

C:\Users\schirmacher3\AppData\Local\Temp\ipykernel_22812\528192847.py:9: RuntimeWarning: overflow encountered in cast
  min_val_gnn = min(min_val_gnn, checkpoint['state_dict'][layer].numpy().min())


est_ro fold4
MIN WEIGHT GNN LAYERS (ALL):  -0.30430362
MAX WEIGHT GNN LAYERS (ALL):  0.29824704
MEAN WEIGHT GNN LAYERS (ALL):  -0.0002519874
STD WEIGHT GNN LAYERS (ALL):  0.12402561
MIN WEIGHT MLP LAYER:  -0.04204539
MAX WEIGHT MLP LAYER:  0.037276816
MEAN WEIGHT MLP LAYER:  -0.0006602316
STD WEIGHT MLP LAYER:  0.0052512647


est_ro fold0
MIN WEIGHT GNN LAYERS (ALL):  -0.29612163
MAX WEIGHT GNN LAYERS (ALL):  0.29810414
MEAN WEIGHT GNN LAYERS (ALL):  0.004117119
STD WEIGHT GNN LAYERS (ALL):  0.14727849
MIN WEIGHT MLP LAYER:  -0.009706846
MAX WEIGHT MLP LAYER:  0.009346587
MEAN WEIGHT MLP LAYER:  -0.0001189682
STD WEIGHT MLP LAYER:  0.0036470848


est_ro fold1
MIN WEIGHT GNN LAYERS (ALL):  -0.29671067
MAX WEIGHT GNN LAYERS (ALL):  0.29761913
MEAN WEIGHT GNN LAYERS (ALL):  -0.0014973283
STD WEIGHT GNN LAYERS (ALL):  0.14708667
MIN WEIGHT MLP LAYER:  -0.009817095
MAX WEIGHT MLP LAYER:  0.009962662
MEAN WEIGHT MLP LAYER:  -0.00010883135
STD WEIGHT MLP LAYER:  0.0036038046


est_ro fold2
MI

Problem: No real quantification of the grade of learning possible! And the std/mean depend on the number of weights and the initilization -> compare to inital Xavier normalization given distribution

In [112]:
def analysis_checkpoint_gnn_extended(checkpoint, model_name):

    shift_z_first = []
    shift_z_second = []
    var_ratio_first = []
    var_ratio_second = []
    frob_ratio_first = []
    frob_ratio_second = []
    

    if model_name =='GAT' or model_name =='GAT2MLP':
        shift_z_att_first = []
        var_ratio_att_first = []
        frob_ratio_att_first = []

        shift_z_att_second = []
        var_ratio_att_second = []
        frob_ratio_att_second = []

    layer_list = list(checkpoint['state_dict'].keys())
    
    if model_name =='GAT' or model_name =='GAT2MLP':
        layer_list = [layer_list[idx] for idx in range(len(layer_list)) if layer_list[idx][-7:]=='.weight' or layer_list[idx][-8:-4]=='.att']
    else:
        layer_list = [layer_list[idx] for idx in range(len(layer_list)) if layer_list[idx][-7:]=='.weight']

    for idx, layer in enumerate(layer_list):
        if layer[:5]=='first': #first linear layer            
            #Attention layers, 8x8
            if layer[-8:-4]=='.att':
                xavier_var = 2/(8+8)
                xavier_mean = 0
                xavier_frob = np.sqrt((2*8*8)/(8+8))

                weight_mean = checkpoint['state_dict'][layer].mean().numpy()
                weight_var = checkpoint['state_dict'][layer].var().numpy()
                weight_frob = np.linalg.norm(np.squeeze(checkpoint['state_dict'][layer].numpy()), ord='fro') #squeeze for reduction of first extra dim

                var_ratio = weight_var/xavier_var
                shift_z_score = np.abs(weight_mean - xavier_mean)/np.sqrt(xavier_var)
                frob_ratio = weight_frob/xavier_frob

                '''
                print('SHIFT Z-SCORE: ', shift_z_score)
                print('VARIANCE RATIO: ', var_ratio)
                print('FROBENIUS NORM RATIO: ', frob_ratio)
                '''

                shift_z_att_first.append(shift_z_score)
                var_ratio_att_first.append(var_ratio)
                frob_ratio_att_first.append(frob_ratio)
            
            #Weight layer 2->64
            else:
                xavier_var = 2/(2+64)
                xavier_mean = 0
                xavier_frob = np.sqrt((2*2*64)/(2+64))

                weight_mean = checkpoint['state_dict'][layer].mean().numpy()
                weight_var = checkpoint['state_dict'][layer].var().numpy()
                weight_frob = np.linalg.norm(checkpoint['state_dict'][layer].numpy(), ord='fro')

                var_ratio = weight_var/xavier_var
                shift_z_score = np.abs(weight_mean - xavier_mean)/np.sqrt(xavier_var)
                frob_ratio = weight_frob/xavier_frob

                '''
                print('SHIFT Z-SCORE: ', shift_z_score)
                print('VARIANCE RATIO: ', var_ratio)
                print('FROBENIUS NORM RATIO: ', frob_ratio)
                '''

                shift_z_first.append(shift_z_score)
                var_ratio_first.append(var_ratio)
                frob_ratio_first.append(frob_ratio)
            

        elif layer[:6]=='hidden': #second linear layer
            #Attention layers, 8x8
            if layer[-8:-4]=='.att':
                xavier_var = 2/(8+8)
                xavier_mean = 0
                xavier_frob = np.sqrt((2*8*8)/(8+8))

                weight_mean = checkpoint['state_dict'][layer].mean().numpy()
                weight_var = checkpoint['state_dict'][layer].var().numpy()
                weight_frob = np.linalg.norm(np.squeeze(checkpoint['state_dict'][layer].numpy()), ord='fro')

                var_ratio = weight_var/xavier_var
                shift_z_score = np.abs(weight_mean - xavier_mean)/np.sqrt(xavier_var)
                frob_ratio = weight_frob/xavier_frob

                '''
                print('SHIFT Z-SCORE: ', shift_z_score)
                print('VARIANCE RATIO: ', var_ratio)
                print('FROBENIUS NORM RATIO: ', frob_ratio)
                '''

                shift_z_att_second.append(shift_z_score)
                var_ratio_att_second.append(var_ratio)
                frob_ratio_att_second.append(frob_ratio)

            #Weight layer 64->64
            else:
                xavier_var = 2/(64+64)
                xavier_mean = 0
                xavier_frob = np.sqrt((2*64*64)/(64+64))

                weight_mean = checkpoint['state_dict'][layer].mean().numpy()
                weight_var = checkpoint['state_dict'][layer].var().numpy()
                weight_frob = np.linalg.norm(checkpoint['state_dict'][layer].numpy(), ord='fro')

                var_ratio = weight_var/xavier_var
                shift_z_score = np.abs(weight_mean - xavier_mean)/np.sqrt(xavier_var)
                frob_ratio = weight_frob/xavier_frob

                '''
                print('SHIFT Z-SCORE: ', shift_z_score)
                print('VARIANCE RATIO: ', var_ratio)
                print('FROBENIUS NORM RATIO: ', frob_ratio)
                '''

                shift_z_second.append(shift_z_score)
                var_ratio_second.append(var_ratio)
                frob_ratio_second.append(frob_ratio)


        elif layer[:11]=='linear_last':
            mlp_weight = checkpoint['state_dict'][layer].numpy()
            mlp_size = checkpoint['state_dict'][layer].numpy().shape[1]  

            xavier_var = 2/(mlp_size + 1)
            xavier_mean = 0
            xavier_frob = np.sqrt(2*mlp_size*1)/(mlp_size+1)
                                    
            weight_mean = mlp_weight.mean()
            weight_var =mlp_weight.var()
            weight_frob = np.linalg.norm(mlp_weight, ord='fro')

            var_ratio_mlp = weight_var/xavier_var
            shift_z_score_mlp = np.abs(weight_mean - xavier_mean)/np.sqrt(xavier_var)
            frob_ratio_mlp = weight_frob/xavier_frob


        else:
            raise ValueError(
            "Invalid layer name! Layer within model not known"
            )
        

    median_shift_z_first = np.median(shift_z_first) 
    median_var_ratio_first = np.median(var_ratio_first)
    median_frob_ratio_first = np.median(frob_ratio_first)   

    median_shift_z_second = np.median(shift_z_second)   
    median_var_ratio_second = np.median(var_ratio_second)
    median_frob_ratio_second = np.median(frob_ratio_second) 

    '''
    print('Layer list:', layer_list)
    print('Z-SCORE RATIO LIST FIRST LAYER: ', shift_z_first, median_shift_z_first)
    print('VAR RATIO LIST FIRST LAYER: ', var_ratio_first, median_var_ratio_first)
    print('FROB RATIO LIST FIRST LAYER: ', frob_ratio_first, median_frob_ratio_first)

    print('Z-SCORE RATIO LIST SECOND LAYER: ', shift_z_second, median_shift_z_second)
    print('VAR RATIO LIST SECOND LAYER: ', var_ratio_second, median_var_ratio_second)
    print('FROB RATIO LIST SECOND LAYER: ', frob_ratio_second, median_frob_ratio_second)
    '''

    if model_name =='GAT' or model_name =='GAT2MLP':
        median_shift_z_att_first = np.median(shift_z_att_first) 
        median_var_ratio_att_first = np.median(var_ratio_att_first) 
        median_frob_ratio_att_first = np.median(frob_ratio_att_first)   

        median_shift_z_att_second = np.median(shift_z_att_second)   
        median_var_ratio_att_second = np.median(var_ratio_att_second)   
        median_frob_ratio_att_second = np.median(frob_ratio_att_second) 

        '''
        print('ATT Z-SCORE RATIO LIST FIRST LAYER: ', shift_z_att_first, median_shift_z_att_first)
        print('ATT VAR RATIO LIST FIRST LAYER: ', var_ratio_att_first, median_var_ratio_att_first)
        print('ATT FROB RATIO LIST FIRST LAYER: ', frob_ratio_att_first, median_frob_ratio_att_first)

        print('Z-SCORE RATIO LIST SECOND LAYER: ', shift_z_att_second, median_shift_z_att_second)
        print('ATT VAR RATIO LIST SECOND LAYER: ', var_ratio_att_second, median_var_ratio_att_second)
        print('ATT FROB RATIO LIST SECOND LAYER: ', frob_ratio_att_second, median_frob_ratio_att_second)
        '''

        return median_shift_z_first, median_var_ratio_first, median_frob_ratio_first, median_shift_z_second, median_var_ratio_second, median_frob_ratio_second, shift_z_score_mlp, var_ratio_mlp, frob_ratio_mlp, median_shift_z_att_first, median_var_ratio_att_first, median_frob_ratio_att_first, median_shift_z_att_second, median_var_ratio_att_second, median_frob_ratio_att_second


    return median_shift_z_first, median_var_ratio_first, median_frob_ratio_first, median_shift_z_second, median_var_ratio_second, median_frob_ratio_second, shift_z_score_mlp, var_ratio_mlp, frob_ratio_mlp




In [113]:
def run_statistics_analysis(path, model_name):

    median_shift_z_first_list = [] 
    median_var_ratio_first_list  = [] 
    median_frob_ratio_first_list  = [] 
    median_shift_z_second_list  = [] 
    median_var_ratio_second_list  = [] 
    median_frob_ratio_second_list  = [] 

    if model_name == 'GAT' or model_name == 'GAT2MLP':
        median_shift_z_att_first_list = [] 
        median_var_ratio_att_first_list = [] 
        median_frob_ratio_att_first_list = [] 
        median_shift_z_att_second_list = [] 
        median_var_ratio_att_second_list = [] 
        median_frob_ratio_att_second_list = [] 
        

    shift_z_mlp_list = [] 
    var_ratio_mlp_list  = [] 
    frob_ratio_mlp_list  = [] 

    for path, subdirs, files in os.walk(path):
        for name in files:
            file_path = os.path.join(path, name).replace("\\","/")
            ckpt = torch.load(file_path, map_location=torch.device('cpu'))

            if model_name == 'GAT' or model_name == 'GAT2MLP':
                median_shift_z_first, median_var_ratio_first, median_frob_ratio_first, median_shift_z_second, median_var_ratio_second, median_frob_ratio_second, shift_z_score_mlp, var_ratio_mlp, frob_ratio_mlp, median_shift_z_att_first, median_var_ratio_att_first, median_frob_ratio_att_first, median_shift_z_att_second, median_var_ratio_att_second, median_frob_ratio_att_second = analysis_checkpoint_gnn_extended(ckpt, model_name)
            else: 
                median_shift_z_first, median_var_ratio_first, median_frob_ratio_first, median_shift_z_second, median_var_ratio_second, median_frob_ratio_second, shift_z_score_mlp, var_ratio_mlp, frob_ratio_mlp = analysis_checkpoint_gnn_extended(ckpt, model_name)


            median_shift_z_first_list.append(median_shift_z_first)
            median_var_ratio_first_list.append(median_var_ratio_first)
            median_frob_ratio_first_list.append(median_frob_ratio_first)

            median_shift_z_second_list.append(median_shift_z_second)
            median_var_ratio_second_list.append(median_var_ratio_second)
            median_frob_ratio_second_list.append(median_frob_ratio_second)

            shift_z_mlp_list.append(shift_z_score_mlp)
            var_ratio_mlp_list.append(var_ratio_mlp)
            frob_ratio_mlp_list.append(frob_ratio_mlp)

            if model_name == 'GAT' or model_name == 'GAT2MLP':
                median_shift_z_att_first_list.append(median_shift_z_att_first)
                median_var_ratio_att_first_list.append(median_var_ratio_att_first)
                median_frob_ratio_att_first_list.append(median_frob_ratio_att_first)

                median_shift_z_att_second_list.append(median_shift_z_att_second)
                median_var_ratio_att_second_list.append(median_var_ratio_att_second)
                median_frob_ratio_att_second_list.append(median_frob_ratio_att_second)


    mean_shift_z_first = sum(median_shift_z_first_list)/len(median_shift_z_first_list)
    mean_var_ratio_first = sum(median_var_ratio_first_list)/len(median_var_ratio_first_list)
    mean_frob_ratio_first = sum(median_frob_ratio_first_list)/len(median_frob_ratio_first_list)

    mean_shift_z_second = sum(median_shift_z_second_list)/len(median_shift_z_second_list)
    mean_var_ratio_second = sum(median_var_ratio_second_list)/len(median_var_ratio_second_list)
    mean_frob_ratio_second = sum(median_frob_ratio_second_list)/len(median_frob_ratio_second_list)

    mean_shift_z_mlp = sum(shift_z_mlp_list)/len(shift_z_mlp_list)
    mean_var_ratio_mlp = sum(var_ratio_mlp_list)/len(var_ratio_mlp_list)
    mean_frob_ratio_mlp = sum(frob_ratio_mlp_list)/len(frob_ratio_mlp_list)

    if model_name == 'GAT' or model_name == 'GAT2MLP':
        mean_shift_z_att_first = sum(median_shift_z_att_first_list)/len(median_shift_z_att_first_list)
        mean_var_ratio_att_first = sum(median_var_ratio_att_first_list)/len(median_var_ratio_att_first_list)
        mean_frob_ratio_att_first = sum(median_frob_ratio_att_first_list)/len(median_frob_ratio_att_first_list)

        mean_shift_z_att_second = sum(median_shift_z_att_second_list)/len(median_shift_z_att_second_list)
        mean_var_ratio_att_second = sum(median_var_ratio_att_second_list)/len(median_var_ratio_att_second_list)
        mean_frob_ratio_att_second = sum(median_frob_ratio_att_second_list)/len(median_frob_ratio_att_second_list)


    print('FIRST GNN LAYER:')
    print('MEAN OF MEDIAN SHIFT Z-SCORE first layer gnn: ', mean_shift_z_first)
    print('MEAN OF MEDIAN VARIANCE RATIO first layer gnn: ', mean_var_ratio_first)
    print('MEAN OF MEDIAN FROBENIUS RATIO first layer gnn: ', mean_frob_ratio_first)

    print('\nSECOND GNN LAYER:')
    print('MEAN OF MEDIAN SHIFT Z-SCORE second layer gnn: ', mean_shift_z_second )
    print('MEAN OF MEDIAN VARIANCE RATIO second layer gnn: ', mean_var_ratio_second)
    print('MEAN OF MEDIAN FROBENIUS RATIO second layer gnn: ', mean_frob_ratio_second)

    if model_name == 'GAT' or model_name == 'GAT2MLP':
        print('\nFIRST ATT LAYER:')
        print('MEAN OF MEDIAN SHIFT Z-SCORE first layer att: ', mean_shift_z_att_first)
        print('MEAN OF MEDIAN VARIANCE RATIO first layer att: ', mean_var_ratio_att_first)
        print('MEAN OF MEDIAN FROBENIUS RATIO first layer att: ', mean_frob_ratio_att_first)

        print('\nSECOND ATT LAYER:')
        print('MEAN OF MEDIAN SHIFT Z-SCORE second layer att: ', mean_shift_z_att_second )
        print('MEAN OF MEDIAN VARIANCE RATIO second layer att: ', mean_var_ratio_att_second)
        print('MEAN OF MEDIAN FROBENIUS RATIO second layer att: ', mean_frob_ratio_att_second)


    print('\nFINAL MLP LAYER')
    print('MEDIAN OF SHIFT Z-SCORE mlp: ', mean_shift_z_mlp)
    print('MEDIAN OF VARIANCE RATIO mlp: ', mean_var_ratio_mlp )
    print('MEDIAN OF FROBENIUS RATIO mlp: ', mean_frob_ratio_mlp)

In [114]:
#GCN
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/GCN'

print('FOR GCN: \n')
run_statistics_analysis(path, 'GCN')

FOR GCN: 

FIRST GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer gnn:  0.5792943863852827
MEAN OF MEDIAN VARIANCE RATIO first layer gnn:  24.519234
MEAN OF MEDIAN FROBENIUS RATIO first layer gnn:  4.840550320852738

SECOND GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer gnn:  0.41087089486420153
MEAN OF MEDIAN VARIANCE RATIO second layer gnn:  36.53712
MEAN OF MEDIAN FROBENIUS RATIO second layer gnn:  5.2373392248153685

FINAL MLP LAYER
MEDIAN OF SHIFT Z-SCORE mlp:  4.7288032456402025
MEDIAN OF VARIANCE RATIO mlp:  90.49013
MEDIAN OF FROBENIUS RATIO mlp:  84.46874561727037


In [115]:
#GCN2MLP
print('FOR ONLY 1 SPLIT!')
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/GCN2MLP'

print('FOR GCN2MLP: \n')
run_statistics_analysis(path, 'GCN2MLP')

FOR ONLY 1 SPLIT!
FOR GCN2MLP: 

FIRST GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer gnn:  1.539946349104166
MEAN OF MEDIAN VARIANCE RATIO first layer gnn:  1.921868
MEAN OF MEDIAN FROBENIUS RATIO first layer gnn:  2.068402358672775

SECOND GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer gnn:  0.8547834157943726
MEAN OF MEDIAN VARIANCE RATIO second layer gnn:  2.2683103
MEAN OF MEDIAN FROBENIUS RATIO second layer gnn:  1.7315919399261475

FINAL MLP LAYER
MEDIAN OF SHIFT Z-SCORE mlp:  2.426645820093245
MEDIAN OF VARIANCE RATIO mlp:  187.84251
MEDIAN OF FROBENIUS RATIO mlp:  4445.652792550208


In [116]:
#GAT
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/GAT'

print('FOR GAT: \n')
run_statistics_analysis(path, 'GAT')

FOR GAT: 

FIRST GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer gnn:  0.4940258127016385
MEAN OF MEDIAN VARIANCE RATIO first layer gnn:  22.776922
MEAN OF MEDIAN FROBENIUS RATIO first layer gnn:  4.432745557970973

SECOND GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer gnn:  0.16156982362270356
MEAN OF MEDIAN VARIANCE RATIO second layer gnn:  17.624075
MEAN OF MEDIAN FROBENIUS RATIO second layer gnn:  3.847298456430435

FIRST ATT LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer att:  0.5797243164082074
MEAN OF MEDIAN VARIANCE RATIO first layer att:  31.816557
MEAN OF MEDIAN FROBENIUS RATIO first layer att:  4.706211166798529

SECOND ATT LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer att:  0.8862611015379241
MEAN OF MEDIAN VARIANCE RATIO second layer att:  17.555016
MEAN OF MEDIAN FROBENIUS RATIO second layer att:  3.618279497789566

FINAL MLP LAYER
MEDIAN OF SHIFT Z-SCORE mlp:  0.5990627540179752
MEDIAN OF VARIANCE RATIO mlp:  9.274128
MEDIAN OF FROBENIUS RATIO mlp:  22.2422

In [117]:
#GAT2MLP
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/GAT2MLP'

print('FOR GAT2MLP: \n')
run_statistics_analysis(path, 'GAT2MLP')

FOR GAT2MLP: 

FIRST GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer gnn:  0.04872440323022148
MEAN OF MEDIAN VARIANCE RATIO first layer gnn:  0.71464944
MEAN OF MEDIAN FROBENIUS RATIO first layer gnn:  0.8421669484239293

SECOND GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer gnn:  0.01085622344398871
MEAN OF MEDIAN VARIANCE RATIO second layer gnn:  0.7930849
MEAN OF MEDIAN FROBENIUS RATIO second layer gnn:  0.883623149394989

FIRST ATT LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer att:  0.05202858227756172
MEAN OF MEDIAN VARIANCE RATIO first layer att:  0.368554
MEAN OF MEDIAN FROBENIUS RATIO first layer att:  0.5533463684754918

SECOND ATT LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer att:  0.045734091676917846
MEAN OF MEDIAN VARIANCE RATIO second layer att:  0.2845141
MEAN OF MEDIAN FROBENIUS RATIO second layer att:  0.4616023763549117

FINAL MLP LAYER
MEDIAN OF SHIFT Z-SCORE mlp:  0.06184391376808283
MEDIAN OF VARIANCE RATIO mlp:  12.967798
MEDIAN OF FROBENIUS RATIO 

In [118]:
#path = '/Users/julia/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet' #old
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet'

print('FOR CHEBNET: \n')
run_statistics_analysis(path, 'ChebNet')

FOR CHEBNET: 

FIRST GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer gnn:  0.330328159152576
MEAN OF MEDIAN VARIANCE RATIO first layer gnn:  2.0859737
MEAN OF MEDIAN FROBENIUS RATIO first layer gnn:  1.4516822279727524

SECOND GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer gnn:  0.26830990379676223
MEAN OF MEDIAN VARIANCE RATIO second layer gnn:  6.734652
MEAN OF MEDIAN FROBENIUS RATIO second layer gnn:  2.4191268894076345

FINAL MLP LAYER
MEDIAN OF SHIFT Z-SCORE mlp:  0.7867369604902938
MEDIAN OF VARIANCE RATIO mlp:  9.174921
MEDIAN OF FROBENIUS RATIO mlp:  24.947177257757286


In [119]:
#path = '/Users/julia/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet2MLP' #old
path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet2MLP'

print('FOR CHEBNET2MLP: \n')
run_statistics_analysis(path, 'ChebNet2MLP')

FOR CHEBNET2MLP: 

FIRST GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE first layer gnn:  0.07822856727513823
MEAN OF MEDIAN VARIANCE RATIO first layer gnn:  0.93196
MEAN OF MEDIAN FROBENIUS RATIO first layer gnn:  0.9670335924595865

SECOND GNN LAYER:
MEAN OF MEDIAN SHIFT Z-SCORE second layer gnn:  0.10666634743800386
MEAN OF MEDIAN VARIANCE RATIO second layer gnn:  0.78390944
MEAN OF MEDIAN FROBENIUS RATIO second layer gnn:  0.8971142852306366

FINAL MLP LAYER
MEDIAN OF SHIFT Z-SCORE mlp:  0.8857898063021831
MEDIAN OF VARIANCE RATIO mlp:  17.09347
MEDIAN OF FROBENIUS RATIO mlp:  1005.9195907692009


Use GNNExplainer to gain more insights ...

In [15]:
def get_model(model_name, ckpt):    
    if model_name =='GCN':
        model = GCN(
                    number_input_channels=ckpt['hyper_parameters']['number_input_channels'],
                    number_hidden_layers=ckpt['hyper_parameters']['number_hidden_layers'],
                    width=ckpt['hyper_parameters']['width'],
                    output_dim=64,
                    learning_rate=ckpt['hyper_parameters']['learning_rate'],
                    weight_decay=ckpt['hyper_parameters']['weight_decay'],
                    dropout_rate=ckpt['hyper_parameters']['dropout_rate'],
                    residual=ckpt['hyper_parameters']['residual'],
                    dense=ckpt['hyper_parameters']['dense'],
                    width_hidden_reduced_dense=ckpt['hyper_parameters']['width_hidden_reduced_dense']
                )
        
    elif model_name == 'GCN2MLP':
        model = GCN2MLP(
                    number_input_channels=ckpt['hyper_parameters']['number_input_channels'],
                    number_input_nodes=ckpt['hyper_parameters']['number_input_nodes'],
                    number_hidden_layers=ckpt['hyper_parameters']['number_hidden_layers'],
                    width=ckpt['hyper_parameters']['width'],
                    output_dim=64,
                    learning_rate=ckpt['hyper_parameters']['learning_rate'],
                    weight_decay=ckpt['hyper_parameters']['weight_decay'],
                    dropout_rate=ckpt['hyper_parameters']['dropout_rate'],
                    residual=ckpt['hyper_parameters']['residual'],
                    dense=ckpt['hyper_parameters']['dense'],
                    width_hidden_reduced_dense=ckpt['hyper_parameters']['width_hidden_reduced_dense']
                )
        
    elif model_name =='GAT':
        model = GAT(
                    number_input_channels=ckpt['hyper_parameters']['number_input_channels'],
                    number_hidden_layers=ckpt['hyper_parameters']['number_hidden_layers'],
                    heads = ckpt['hyper_parameters']['heads'],
                    width=ckpt['hyper_parameters']['width'],
                    output_dim=64,
                    learning_rate=ckpt['hyper_parameters']['learning_rate'],
                    weight_decay=ckpt['hyper_parameters']['weight_decay'],
                    dropout_rate_normal=ckpt['hyper_parameters']['dropout_rate_normal'],
                    dropout_rate_attention=ckpt['hyper_parameters']['dropout_rate_attention'],
                    residual=ckpt['hyper_parameters']['residual'],
                    dense=ckpt['hyper_parameters']['dense'],
                    width_hidden_reduced_dense=ckpt['hyper_parameters']['width_hidden_reduced_dense']
                )
        
    elif model_name == 'GAT2MLP':
        model = GAT2MLP(
                    number_input_channels=ckpt['hyper_parameters']['number_input_channels'],
                    number_input_nodes=ckpt['hyper_parameters']['number_input_nodes'],
                    number_hidden_layers=ckpt['hyper_parameters']['number_hidden_layers'],
                    heads = ckpt['hyper_parameters']['heads'],
                    width=ckpt['hyper_parameters']['width'],
                    output_dim=64,
                    learning_rate=ckpt['hyper_parameters']['learning_rate'],
                    weight_decay=ckpt['hyper_parameters']['weight_decay'],
                    dropout_rate_normal=ckpt['hyper_parameters']['dropout_rate_normal'],
                    dropout_rate_attention=ckpt['hyper_parameters']['dropout_rate_attention'],
                    residual=ckpt['hyper_parameters']['residual'],
                    dense=ckpt['hyper_parameters']['dense'],
                    width_hidden_reduced_dense=ckpt['hyper_parameters']['width_hidden_reduced_dense']
                )

    elif model_name == 'ChebNet':
        model = ChebNet(
                        number_input_channels=ckpt['hyper_parameters']['number_input_channels'],
                        number_hidden_layers=ckpt['hyper_parameters']['number_hidden_layers'],
                        width=ckpt['hyper_parameters']['width'],
                        output_dim=ckpt['hyper_parameters']['output_dim'],
                        K=ckpt['hyper_parameters']['K'],
                        learning_rate=ckpt['hyper_parameters']['learning_rate'],
                        weight_decay=ckpt['hyper_parameters']['weight_decay'],
                        dropout_rate=ckpt['hyper_parameters']['dropout_rate'],
                        residual=ckpt['hyper_parameters']['residual'],
                        dense=ckpt['hyper_parameters']['dense'],
                        width_hidden_reduced_dense=ckpt['hyper_parameters']['width_hidden_reduced_dense']
                )
        
    elif model_name == 'ChebNet2MLP':
        model = ChebNet2MLP(
                        number_input_channels=ckpt['hyper_parameters']['number_input_channels'],
                        number_input_nodes=ckpt['hyper_parameters']['number_input_nodes'],
                        number_hidden_layers=ckpt['hyper_parameters']['number_hidden_layers'],
                        width=ckpt['hyper_parameters']['width'],
                        output_dim=ckpt['hyper_parameters']['output_dim'],
                        K=ckpt['hyper_parameters']['K'],
                        learning_rate=ckpt['hyper_parameters']['learning_rate'],
                        weight_decay=ckpt['hyper_parameters']['weight_decay'],
                        dropout_rate=ckpt['hyper_parameters']['dropout_rate'],
                        residual=ckpt['hyper_parameters']['residual'],
                        dense=ckpt['hyper_parameters']['dense'],
                        width_hidden_reduced_dense=ckpt['hyper_parameters']['width_hidden_reduced_dense']
                )
                    

    else:
        raise ValueError('Model name not known! Check your model name and choose one of [GCN, GCN2MLP, GAT, GAT2MLP, ChebNet, ChebNet2MLP]')
    
    return model

In [16]:
# EDGE IMPORTANCES
def edge_importance(model_name, test_data, ckpt, epochs=200):
    test_data_edges = test_data[0].edge_index #all graphs have same edges

    edge_mask_mean_list = []
    edge_mask_var_list = []
    edge_mask_range_list = []
    edge_mask_entropy_list = []

    for idx in tqdm(range(len(test_data))):
          test_data_x = test_data[idx].x
          test_data_y = test_data[idx].y

          model = get_model(model_name, ckpt)
          model.eval()

          explainer = Explainer(
               model = model,
               algorithm = GNNExplainer(epochs=epochs),
               explanation_type = 'model',
               model_config = dict(
                    mode = 'binary_classification',
                    task_level = 'graph',
                    return_type = 'raw'
               ),
               node_mask_type = None,
               edge_mask_type = 'object'
          )
          
          explanation = explainer(test_data_x, test_data_edges,batch_assign=None)

          #pred = torch.round(torch.sigmoid(explanation.prediction))
          #masked_pred = torch.round(torch.sigmoid(explainer.get_masked_prediction(test_data_x, test_data_edges,batch_assign=None)))

          edge_mask = explanation.edge_mask
          edge_mask_mean = explanation.edge_mask.mean()
          edge_mask_var = explanation.edge_mask.var()
          edge_mask_max = explanation.edge_mask.max()
          edge_mask_min = explanation.edge_mask.min()
          edge_mask_entropy = scipy.stats.entropy(edge_mask)
          edge_mask_range = edge_mask_max - edge_mask_min

          '''
          print('Edge mask:', edge_mask)
          print('Edge mask mean, var, min, max: ', edge_mask_mean, edge_mask_var, edge_mask_min, edge_mask_max)
          print('Range', edge_mask_range)
          print('Entropy: ', edge_mask_entropy)
          '''

          #print('Prediction true? ',(pred == test_data_y).item())
          #print('Masked prediction true? ',(masked_pred == test_data_y).item())
          #print('Prediction: ', pred)
          #print('Prediction Explainer (Before masking): ', explanation.prediction)
          #print('Masked prediction: ', masked_pred)
          #print('Prediction Explainer (After masking): ', explainer.get_masked_prediction(test_data_x, test_data_edges,batch_assign=None))
          #print(explainer.get_masked_prediction(test_data_x, test_data_edges,batch_assign=None))
          #print('Ground truth: ', test_data_y)
          #print('Difference prediction vs. masked prediction: ', abs(explanation.prediction - explainer.get_masked_prediction(test_data_x, test_data_edges,batch_assign=None)))

          edge_mask_mean_list.append(edge_mask_mean)        
          edge_mask_var_list.append(edge_mask_var)
          edge_mask_range_list.append(edge_mask_range)
          edge_mask_entropy_list.append(edge_mask_entropy)


    mean_edge_mask_mean = sum(edge_mask_mean_list)/len(edge_mask_mean_list)
    mean_edge_mask_var = sum(edge_mask_var_list)/len(edge_mask_var_list)
    mean_edge_mask_range = sum(edge_mask_range_list)/len(edge_mask_range_list)
    mean_edge_mask_entropy = sum(edge_mask_entropy_list)/len(edge_mask_entropy_list)
    
    print('MEAN EDGE MASK MEAN: ', mean_edge_mask_mean)
    print('MEAN EDGE MASK VAR: ', mean_edge_mask_var)
    print('MEAN EDGE MASK RANGE: ', mean_edge_mask_range)
    print('MEAN EDGE MASK ENTROPY: ', mean_edge_mask_entropy)

    return mean_edge_mask_mean, mean_edge_mask_var, mean_edge_mask_range, mean_edge_mask_entropy


In [17]:
# NODE IMPORTANCES (WHOLE NODES)
def node_importance(model_name, test_data, ckpt, epochs=200):
    test_data_edges = test_data[0].edge_index #all graphs have same edges

    node_mask_mean_list = []
    node_mask_var_list = []
    node_mask_range_list = []
    node_mask_entropy_list = []

    for idx in tqdm(range(len(test_data))):
        test_data_x = test_data[idx].x
        test_data_y = test_data[idx].y

        model = get_model(model_name, ckpt)
        model.eval()

        explainer = Explainer(
            model = model,
            algorithm = GNNExplainer(epochs=epochs),
            explanation_type = 'model',
            model_config = dict(
                mode = 'binary_classification',
                task_level = 'graph',
                return_type = 'raw'
            ),
            node_mask_type = 'object',
            edge_mask_type = None
        )

        explanation = explainer(test_data_x, test_data_edges,batch_assign=None)

        node_mask = explanation.node_mask
        node_mask_mean = explanation.node_mask.mean()
        node_mask_var = explanation.node_mask.var()
        node_mask_max = explanation.node_mask.max()
        node_mask_min = explanation.node_mask.min()
        node_mask_entropy = scipy.stats.entropy(node_mask)
        node_mask_range = node_mask_max - node_mask_min


        #pred = torch.round(torch.sigmoid(explanation.prediction))
        #masked_pred = torch.round(torch.sigmoid(explainer.get_masked_prediction(test_data_x, test_data_edges,batch_assign=None)))

        '''
        print('Node mask:',node_mask.T)
        print('Node mask mean, var, min, max: ',node_mask_mean, node_mask_var, node_mask_min, node_mask_max)
        '''

        node_mask_mean_list.append(node_mask_mean)        
        node_mask_var_list.append(node_mask_var)
        node_mask_range_list.append(node_mask_range)
        node_mask_entropy_list.append(node_mask_entropy)
     

    mean_node_mask_mean = sum(node_mask_mean_list)/len(node_mask_mean_list)
    mean_node_mask_var = sum(node_mask_var_list)/len(node_mask_var_list)
    mean_node_mask_range = sum(node_mask_range_list)/len(node_mask_range_list)
    mean_node_mask_entropy = sum(node_mask_entropy_list)/len(node_mask_entropy_list)
    
    print('MEAN NODE MASK MEAN: ', mean_node_mask_mean)
    print('MEAN NODE MASK VAR: ', mean_node_mask_var)
    print('MEAN NODE MASK RANGE: ', mean_node_mask_range)
    print('MEAN NODE MASK ENTROPY: ', mean_node_mask_entropy)

    return mean_node_mask_mean, mean_node_mask_var, mean_node_mask_range, mean_node_mask_entropy

In [18]:
# NODE IMPORTANCES (FEATURES, ALL THE SAME ACROSS ALL NODES)
def feature_importance(model_name, test_data, ckpt, epochs=200):
    test_data_edges = test_data[0].edge_index #all graphs have same edges

    x_mean_list = []
    m_mean_list = []


    for idx in tqdm(range(len(test_data))):
        test_data_x = test_data[idx].x
        test_data_y = test_data[idx].y
        
        model = get_model(model_name, ckpt)
        model.eval()

        explainer = Explainer(
            model = model,
            algorithm = GNNExplainer(epochs=epochs),
            explanation_type = 'model',
            model_config = dict(
                mode = 'binary_classification',
                task_level = 'graph',
                return_type = 'raw'
            ),
            node_mask_type = 'common_attributes',
            edge_mask_type = None
        )

        explanation = explainer(test_data_x, test_data_edges,batch_assign=None)

        x_mask = explanation.node_mask[0][0]
        m_mask = explanation.node_mask[0][1]

        #pred = torch.round(torch.sigmoid(explanation.prediction))
        #masked_pred = torch.round(torch.sigmoid(explainer.get_masked_prediction(test_data_x, test_data_edges,batch_assign=None)))

        #print('Node mask:', x_mask, m_mask)

        x_mean_list.append(x_mask)
        m_mean_list.append(m_mask)

    mean_x_mask = sum(x_mean_list)/len(x_mean_list)
    mean_m_mask = sum(m_mean_list)/len(m_mean_list)

    print('MEAN X MASK: ', mean_x_mask)
    print('MEAN M MASK: ', mean_m_mask)

    return mean_x_mask, mean_m_mask

In [19]:
def run_importance_analysis(model_name, base_path, mode):
    number_test_rounds = 10
    number_es_rounds = 5
    round_counter = 0

    if mode == 'edge' or mode == 'node':
        mean_mask_mean_list = []
        mean_mask_var_list = []
        mean_mask_range_list = []
        mean_mask_entropy_list = []

    elif mode == 'feature':
        mean_x_mean_list = []
        mean_m_mean_list = []

    else:
        print('MODE NOT IMPLEMENTED!')
        return        

    #load data
    data_dir = base_path +'/data/kirc_random_nodes_preprocessed_all.pkl'
    with open(data_dir, "rb") as f:
        data = KIRCDataset(pickle.load(f))

    # load data split information
    data_split = torch.load(base_path + '/data/data_splits_KIRC.pt', weights_only=False)

    for path, subdirs, files in os.walk(base_path+'/final_results/KIRC/checkpoints/'+model_name):
        for name in files:
                # Load checkpoint, i.e. model
                file_path = os.path.join(path, name).replace("\\","/")
                ckpt = torch.load(file_path, map_location=torch.device('cpu'))

                # Compute round number
                es_round = round_counter%5
                test_round = int((round_counter-es_round)/5)
                round_counter+=1

                print(f'TEST ROUND {test_round}, ES FOLD {es_round}')

                # Get test data indices & load test data
                outer_test_idx = data_split["outer_test_idx"]
                test_idx = outer_test_idx[test_round]
                test_idx = test_idx[test_idx >= 0]

                test_data = torch.utils.data.Subset(data, test_idx)

                # Apply importance
                if mode == 'edge':
                        mean_mask_mean, mean_mask_var, mean_mask_range, mean_mask_entropy = edge_importance(model_name, test_data, ckpt)

                elif mode == 'node':
                        mean_mask_mean, mean_mask_var, mean_mask_range, mean_mask_entropy = node_importance(model_name, test_data, ckpt)

                elif mode == 'feature':
                       mean_x_mask, mean_m_mask = feature_importance(model_name, test_data, ckpt)
                       

                if mode == 'edge' or mode == 'node':
                    mean_mask_mean_list.append(mean_mask_mean)
                    mean_mask_var_list.append(mean_mask_var)
                    mean_mask_range_list.append(mean_mask_range)
                    mean_mask_entropy_list.append(mean_mask_entropy)

                elif mode == 'feature':
                    mean_x_mean_list.append(mean_x_mask)
                    mean_m_mean_list.append(mean_m_mask)
                      

    if mode == 'edge' or mode == 'node':
        mean_mean_mask_mean = sum(mean_mask_mean_list)/len(mean_mask_mean_list)
        mean_mean_mask_var= sum(mean_mask_var_list)/len(mean_mask_var_list)
        mean_mean_mask_range = sum(mean_mask_range_list)/len(mean_mask_range_list)
        mean_mean_mask_entropy = sum(mean_mask_entropy_list)/len(mean_mask_entropy_list)

        print('MEAN OF MASK MEAN: ', mean_mean_mask_mean)
        print('MEAN OF MASK VARIANCE: ', mean_mean_mask_var)
        print('MEAN OF MASK RANGE: ', mean_mean_mask_range)
        print('MEAN OF MASK ENTROPY: ', mean_mean_mask_entropy)              

        return mean_mean_mask_mean, mean_mean_mask_var, mean_mean_mask_range, mean_mean_mask_entropy

    elif mode == 'feature':
        mean_x_mean = sum(mean_x_mean_list)/len(mean_x_mean_list)
        mean_m_mean = sum(mean_m_mean_list)/len(mean_m_mean_list)

        print('MEAN EXPRESSION MASK: ', mean_x_mean)
        print('MEAN METHYLATION MASK: ', mean_m_mean)

        return mean_x_mean, mean_m_mean
        

In [20]:
#GCN EDGE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gcn_mean_edge_mask_mean, gcn_mean_edge_mask_var, gcn_mean_edge_mask_range, gcn_mean_edge_mask_entropy = run_importance_analysis('GCN', base_path, 'edge')


TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [03:01<00:00,  1.78s/it]


MEAN EDGE MASK MEAN:  tensor(0.1472)
MEAN EDGE MASK VAR:  tensor(1.1490e-05)
MEAN EDGE MASK RANGE:  tensor(0.0251)
MEAN EDGE MASK ENTROPY:  8.540647
TEST ROUND 0, ES FOLD 1


100%|██████████| 102/102 [03:18<00:00,  1.95s/it]


MEAN EDGE MASK MEAN:  tensor(0.1472)
MEAN EDGE MASK VAR:  tensor(1.1487e-05)
MEAN EDGE MASK RANGE:  tensor(0.0251)
MEAN EDGE MASK ENTROPY:  8.540647
TEST ROUND 0, ES FOLD 2


...

MEAN EDGE MASK MEAN:  tensor(0.1472)
MEAN EDGE MASK VAR:  tensor(1.1510e-05)
MEAN EDGE MASK RANGE:  tensor(0.0250)
MEAN EDGE MASK ENTROPY:  8.540647
TEST ROUND 9, ES FOLD 3


100%|██████████| 102/102 [02:58<00:00,  1.75s/it]


MEAN EDGE MASK MEAN:  tensor(0.1472)
MEAN EDGE MASK VAR:  tensor(1.1487e-05)
MEAN EDGE MASK RANGE:  tensor(0.0251)
MEAN EDGE MASK ENTROPY:  8.540647
TEST ROUND 9, ES FOLD 4


100%|██████████| 102/102 [02:54<00:00,  1.72s/it]

MEAN EDGE MASK MEAN:  tensor(0.1472)
MEAN EDGE MASK VAR:  tensor(1.1476e-05)
MEAN EDGE MASK RANGE:  tensor(0.0250)
MEAN EDGE MASK ENTROPY:  8.5406475
MEAN OF MASK MEAN:  tensor(0.1472)
MEAN OF MASK VARIANCE:  tensor(1.1513e-05)
MEAN OF MASK RANGE:  tensor(0.0250)
MEAN OF MASK ENTROPY:  8.540648


In [139]:
#GCN NODE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gcn_mean_node_mask_mean, gcn_mean_node_mask_var, gcn_mean_node_mask_range, gcn_mean_node_mask_entropy = run_importance_analysis('GCN', base_path, 'node')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [03:23<00:00,  1.99s/it]


MEAN NODE MASK MEAN:  tensor(0.1432)
MEAN NODE MASK VAR:  tensor(8.3482e-05)
MEAN NODE MASK RANGE:  tensor(0.0609)
MEAN NODE MASK ENTROPY:  [7.3719673]
TEST ROUND 0, ES FOLD 1


 14%|█▎        | 14/102 [00:33<03:32,  2.42s/it]


KeyboardInterrupt: 

In [140]:
#GCN FEATURE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gcn_mean_x_mean, gcn_mean_m_mean = run_importance_analysis('GCN', base_path, 'feature')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [02:47<00:00,  1.64s/it]


MEAN X MASK:  tensor(0.1429)
MEAN M MASK:  tensor(0.1435)
TEST ROUND 0, ES FOLD 1


  1%|          | 1/102 [00:02<03:32,  2.10s/it]


KeyboardInterrupt: 

In [99]:
#GCN2MLP EDGE
print('FOR ONLY 1 SPLIT!')
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gcn2mlp_mean_edge_mask_mean, gcn2mlp_mean_edge_mask_var, gcn2mlp_mean_edge_mask_range, gcn2mlp_mean_edge_mask_entropy = run_importance_analysis('GCN2MLP', base_path, 'edge')

FOR ONLY 1 SPLIT!
TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [03:28<00:00,  2.04s/it]

MEAN EDGE MASK MEAN:  tensor(0.1465)
MEAN EDGE MASK VAR:  tensor(1.1784e-05)
MEAN EDGE MASK RANGE:  tensor(0.0252)
MEAN EDGE MASK ENTROPY:  8.540642
MEAN OF MASK MEAN:  tensor(0.1465)
MEAN OF MASK VARIANCE:  tensor(1.1784e-05)
MEAN OF MASK RANGE:  tensor(0.0252)
MEAN OF MASK ENTROPY:  tensor(0.0252)


In [100]:
#GCN2MLP NODE
print('FOR ONLY 1 SPLIT!')
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gcn2mlp_mean_node_mask_mean, gcn2mlp_mean_node_mask_var, gcn2mlp_mean_node_mask_range, gcn2mlp_mean_node_mask_entropy = run_importance_analysis('GCN2MLP', base_path, 'node')

FOR ONLY 1 SPLIT!
TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [02:20<00:00,  1.38s/it]

MEAN NODE MASK MEAN:  tensor(0.1739)
MEAN NODE MASK VAR:  tensor(0.0194)
MEAN NODE MASK RANGE:  tensor(0.7651)
MEAN NODE MASK ENTROPY:  [7.201926]
MEAN OF MASK MEAN:  tensor(0.1739)
MEAN OF MASK VARIANCE:  tensor(0.0194)
MEAN OF MASK RANGE:  tensor(0.7651)
MEAN OF MASK ENTROPY:  tensor(0.7651)


In [101]:
#GCN2MLP FEATURE
print('FOR ONLY 1 SPLIT!')
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gcn2mlp_mean_x_mean, gcn2mlp_mean_m_mean = run_importance_analysis('GCN2MLP', base_path, 'feature')

FOR ONLY 1 SPLIT!
TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [02:15<00:00,  1.33s/it]

MEAN X MASK:  tensor(0.1435)
MEAN M MASK:  tensor(0.1433)
MEAN EXPRESSION MASK:  tensor(0.1435)
MEAN METHYLATION MASK:  tensor(0.1433)


In [102]:
#GAT EDGE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gat_mean_edge_mask_mean, gat_mean_edge_mask_var, gat_mean_edge_mask_range, gat_mean_edge_mask_entropy = run_importance_analysis('GAT', base_path, 'edge')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [28:44<00:00, 16.91s/it]


MEAN EDGE MASK MEAN:  tensor(0.1472)
MEAN EDGE MASK VAR:  tensor(1.1532e-05)
MEAN EDGE MASK RANGE:  tensor(0.0252)
MEAN EDGE MASK ENTROPY:  8.540646
TEST ROUND 0, ES FOLD 1


  0%|          | 0/102 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [103]:
#GAT NODE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gat_mean_node_mask_mean, gat_mean_node_mask_var, gat_mean_node_mask_range, gat_mean_node_mask_entropy = run_importance_analysis('GAT', base_path, 'node')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [20:53<00:00, 12.29s/it]


MEAN NODE MASK MEAN:  tensor(0.1433)
MEAN NODE MASK VAR:  tensor(0.0001)
MEAN NODE MASK RANGE:  tensor(0.0938)
MEAN NODE MASK ENTROPY:  [7.371614]
TEST ROUND 0, ES FOLD 1


  3%|▎         | 3/102 [00:36<20:08, 12.21s/it]


KeyboardInterrupt: 

In [111]:
#GAT FEATURE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gat_mean_x_mean, gat_mean_m_mean = run_importance_analysis('GAT', base_path, 'feature')

TEST ROUND 0, ES FOLD 0


  1%|          | 1/102 [00:09<16:18,  9.69s/it]

Node mask: tensor(0.1385) tensor(0.1326)


  2%|▏         | 2/102 [00:19<15:59,  9.60s/it]

Node mask: tensor(0.1294) tensor(0.1410)


...

Node mask: tensor(0.1551) tensor(0.1376)


  4%|▍         | 4/102 [00:38<15:41,  9.61s/it]

Node mask: tensor(0.1304) tensor(0.1627)


  4%|▍         | 4/102 [00:42<17:30, 10.72s/it]


KeyboardInterrupt: 

In [114]:
#GAT2MLP EDGE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gat2mlp_mean_edge_mask_mean, gat2mlp_mean_edge_mask_var, gat2mlp_mean_edge_mask_range, gat2mlp_mean_edge_mask_entropy = run_importance_analysis('GAT2MLP', base_path, 'edge')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [27:45<00:00, 16.33s/it]


MEAN EDGE MASK MEAN:  tensor(0.1465)
MEAN EDGE MASK VAR:  tensor(1.1792e-05)
MEAN EDGE MASK RANGE:  tensor(0.0256)
MEAN EDGE MASK ENTROPY:  8.540641
TEST ROUND 0, ES FOLD 1


 14%|█▎        | 14/102 [03:49<24:00, 16.37s/it]


KeyboardInterrupt: 

In [115]:
#GAT2MLP NODE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gat2mlp_mean_node_mask_mean, gat2mlp_mean_node_mask_var, gat2mlp_mean_node_mask_range, gat2mlp_mean_node_mask_entropy = run_importance_analysis('GAT2MLP', base_path, 'node')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [20:45<00:00, 12.21s/it]


MEAN NODE MASK MEAN:  tensor(0.1743)
MEAN NODE MASK VAR:  tensor(0.0203)
MEAN NODE MASK RANGE:  tensor(0.7629)
MEAN NODE MASK ENTROPY:  [7.1921697]
TEST ROUND 0, ES FOLD 1


  6%|▌         | 6/102 [01:15<20:08, 12.59s/it]


KeyboardInterrupt: 

In [116]:
#GAT2MLP FEATURE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
gat2mlp_mean_x_mean, gat2mlp_mean_m_mean= run_importance_analysis('GAT2MLP', base_path, 'feature')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [22:29<00:00, 13.23s/it]


MEAN X MASK:  tensor(0.1428)
MEAN M MASK:  tensor(0.1430)
TEST ROUND 0, ES FOLD 1


  0%|          | 0/102 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [117]:
#CHEBNET EDGE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
chebnet_mean_edge_mask_mean, chebnet_mean_edge_mask_var, chebnet_mean_edge_mask_range, chebnet_mean_edge_mask_entropy = run_importance_analysis('ChebNet', base_path, 'edge')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [17:23<00:00, 10.23s/it]


MEAN EDGE MASK MEAN:  tensor(0.1467)
MEAN EDGE MASK VAR:  tensor(1.3969e-05)
MEAN EDGE MASK RANGE:  tensor(0.0287)
MEAN EDGE MASK ENTROPY:  8.540584
TEST ROUND 0, ES FOLD 1


 15%|█▍        | 15/102 [02:22<13:44,  9.48s/it]


KeyboardInterrupt: 

In [118]:
#CHEBNET NODE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
chebnet_mean_node_mask_mean, chebnet_mean_node_mask_var, chebnet_mean_node_mask_range, chebnet_mean_node_mask_entropy = run_importance_analysis('ChebNet', base_path, 'node')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [09:02<00:00,  5.31s/it]


MEAN NODE MASK MEAN:  tensor(0.1493)
MEAN NODE MASK VAR:  tensor(0.0035)
MEAN NODE MASK RANGE:  tensor(0.4956)
MEAN NODE MASK ENTROPY:  [7.339295]
TEST ROUND 0, ES FOLD 1


  0%|          | 0/102 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [119]:
#CHEBNET FEATURE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
chebnet_mean_x_mean, chebnet_mean_m_mean = run_importance_analysis('ChebNet', base_path, 'feature')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [09:33<00:00,  5.62s/it]


MEAN X MASK:  tensor(0.1440)
MEAN M MASK:  tensor(0.1445)
TEST ROUND 0, ES FOLD 1


 17%|█▋        | 17/102 [01:47<08:59,  6.34s/it]


KeyboardInterrupt: 

In [141]:
#CHEBNET2MLP EDGE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
chebnet2mlp_mean_edge_mask_mean, chebnet2mlp_mean_edge_mask_var, chebnet2mlp_mean_edge_mask_range, chebnet2mlp_mean_edge_mask_entropy = run_importance_analysis('ChebNet2MLP', base_path, 'edge')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [17:43<00:00, 10.42s/it]


MEAN EDGE MASK MEAN:  tensor(0.1672)
MEAN EDGE MASK VAR:  tensor(0.0044)
MEAN EDGE MASK RANGE:  tensor(0.7415)
MEAN EDGE MASK ENTROPY:  8.487934
TEST ROUND 0, ES FOLD 1


 25%|██▌       | 26/102 [04:29<13:07, 10.36s/it]


KeyboardInterrupt: 

In [142]:
#CHEBNET2MLP NODE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
chebnet2mlp_mean_node_mask_mean, chebnet2mlp_mean_node_mask_var, chebnet2mlp_mean_node_mask_range, chebnet2mlp_mean_node_mask_entropy = run_importance_analysis('ChebNet2MLP', base_path, 'node')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [09:24<00:00,  5.54s/it]


MEAN NODE MASK MEAN:  tensor(0.3607)
MEAN NODE MASK VAR:  tensor(0.0436)
MEAN NODE MASK RANGE:  tensor(0.7511)
MEAN NODE MASK ENTROPY:  [7.217351]
TEST ROUND 0, ES FOLD 1


  0%|          | 0/102 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [144]:
#CHEBNET2MLP FEATURE
base_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi'
chebnet2mlp_mean_x_mean, chebnet2mlp_mean_m_mean  = run_importance_analysis('ChebNet2MLP', base_path, 'feature')

TEST ROUND 0, ES FOLD 0


100%|██████████| 102/102 [09:13<00:00,  5.43s/it]


MEAN X MASK:  tensor(0.1469)
MEAN M MASK:  tensor(0.1499)
TEST ROUND 0, ES FOLD 1


100%|██████████| 102/102 [09:09<00:00,  5.38s/it]


MEAN X MASK:  tensor(0.1465)
MEAN M MASK:  tensor(0.1448)
TEST ROUND 0, ES FOLD 2


...

MEAN X MASK:  tensor(0.1446)
MEAN M MASK:  tensor(0.1485)
TEST ROUND 0, ES FOLD 3


100%|██████████| 102/102 [08:51<00:00,  5.21s/it]


MEAN X MASK:  tensor(0.1437)
MEAN M MASK:  tensor(0.1482)
TEST ROUND 0, ES FOLD 4


  2%|▏         | 2/102 [00:14<11:46,  7.07s/it]


KeyboardInterrupt: 

Raw for single runs for ChebNet:

In [145]:
#Set path for weight from specific test_round, es_round
file_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet/run_test_round0,lr0.01,wd0.0,dropout0.001,batchsize506,es_fold0_outer/epoch=246-step=247.ckpt'
ckpt = torch.load(file_path, map_location=torch.device('cpu'))
test_round = 0
es_round = 0

#get data splits
pl.seed_everything(100*test_round+es_round)
data_split = torch.load('/Users/schirmacher3/PycharmProjects/gnn-for-ppi/data/data_splits_KIRC.pt', weights_only=False)
outer_test_idx = data_split["outer_test_idx"]
test_idx = outer_test_idx[test_round]
test_idx = test_idx[test_idx >= 0]

#load & select data
data_dir = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/data/kirc_random_nodes_preprocessed_all.pkl'
with open(data_dir, "rb") as f:
            data = KIRCDataset(pickle.load(f))

test_data = torch.utils.data.Subset(data, test_idx)

edge_importance('ChebNet', test_data, ckpt)

Seed set to 0
100%|██████████| 102/102 [17:57<00:00, 10.57s/it]

MEAN EDGE MASK MEAN:  tensor(0.1464)
MEAN EDGE MASK VAR:  tensor(1.3989e-05)
MEAN EDGE MASK RANGE:  tensor(0.0291)
MEAN EDGE MASK ENTROPY:  8.540581


(tensor(0.1464), tensor(1.3989e-05), tensor(0.0291), np.float32(8.540581))

In [146]:
node_importance('ChebNet', test_data, ckpt)

100%|██████████| 102/102 [10:18<00:00,  6.06s/it]

MEAN NODE MASK MEAN:  tensor(0.1502)
MEAN NODE MASK VAR:  tensor(0.0040)
MEAN NODE MASK RANGE:  tensor(0.5281)
MEAN NODE MASK ENTROPY:  [7.334474]


(tensor(0.1502),
 tensor(0.0040),
 tensor(0.5281),
 array([7.334474], dtype=float32))

In [147]:
feature_importance('ChebNet', test_data, ckpt)

100%|██████████| 102/102 [12:44<00:00,  7.50s/it]

MEAN X MASK:  tensor(0.1443)
MEAN M MASK:  tensor(0.1460)


(tensor(0.1443), tensor(0.1460))

In [148]:
#CHEBNET2MLP!!!
# 
#Set path for weight from specific test_round, es_round
file_path = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/final_results/KIRC/checkpoints/ChebNet2MLP/run_test_round0,lr0.01,wd0.0,dropout0.001,batchsize506,es_fold0_outer/epoch=31-step=32.ckpt'
ckpt = torch.load(file_path, map_location=torch.device('cpu'))
test_round = 0
es_round = 0

#get data splits
pl.seed_everything(100*test_round+es_round)
data_split = torch.load('/Users/schirmacher3/PycharmProjects/gnn-for-ppi/data/data_splits_KIRC.pt', weights_only=False)
outer_test_idx = data_split["outer_test_idx"]
test_idx = outer_test_idx[test_round]
test_idx = test_idx[test_idx >= 0]

#load & select data
data_dir = '/Users/schirmacher3/PycharmProjects/gnn-for-ppi/data/kirc_random_nodes_preprocessed_all.pkl'
with open(data_dir, "rb") as f:
            data = KIRCDataset(pickle.load(f))

test_data = torch.utils.data.Subset(data, test_idx)

edge_importance('ChebNet2MLP', test_data, ckpt)

Seed set to 0
100%|██████████| 102/102 [23:12<00:00, 13.65s/it]

MEAN EDGE MASK MEAN:  tensor(0.1675)
MEAN EDGE MASK VAR:  tensor(0.0043)
MEAN EDGE MASK RANGE:  tensor(0.7336)
MEAN EDGE MASK ENTROPY:  8.489224


(tensor(0.1675), tensor(0.0043), tensor(0.7336), np.float32(8.489224))

In [149]:
node_importance('ChebNet2MLP', test_data, ckpt)

100%|██████████| 102/102 [14:18<00:00,  8.41s/it]

MEAN NODE MASK MEAN:  tensor(0.3615)
MEAN NODE MASK VAR:  tensor(0.0436)
MEAN NODE MASK RANGE:  tensor(0.7559)
MEAN NODE MASK ENTROPY:  [7.217679]


(tensor(0.3615),
 tensor(0.0436),
 tensor(0.7559),
 array([7.217679], dtype=float32))

In [150]:
feature_importance('ChebNet2MLP', test_data, ckpt)

100%|██████████| 102/102 [16:24<00:00,  9.65s/it]

MEAN X MASK:  tensor(0.1455)
MEAN M MASK:  tensor(0.1513)


(tensor(0.1455), tensor(0.1513))